### Exploration and Analysis of Shimmer IMU data of MP
In this notebook, we will perform the basic analysis of shimmer data. This data has been obtained from Shimer IMU placed on the different body parts of the participants. There are two sources of data: video data used to capture the execution and sensor data obtained from the sensors attached to the body of the participants.

In this notebook we currently consider the MP exercise. The raw data consists of a single csv file for each type of MP.

- Reading the data is quite easy. We just have to rename the file to .csv extension.
- Description 5 sensors (lower back, 2 wrists, 2 elbows)
- Accelerometer (movement acceleration)
        Calibrated and Uncalibrated
            We have to use the calibrated one
        Low noise and wide range
            These two are approximately the same. We only have to use the LN. 
        X , Y and Z coordinates
- Magnetometer (orientation/compass?)
        Calibrated and Uncalibrated
        X , Y and Z coordinates
- Gyroscope (angular velocity, degrees per second)
        Calibrated and Uncalibrated
        X , Y and Z coordinates
- Magnetometer can be used to do the segmentation
- Each sensor 33 columns
    12 from accelerometer
    6 from magnetometer
    6 from gyroscope
    Other columns
        Batt_percentage - 1
        Event Marker -1 
        Packer Reception Rate Trial
        Packet Reception Rate Current
        SyncTimestamp
        System Timestamp
            Cal and uncal
        Timestamp
            Cal and uncal
- We don’t have to use the uncal data, only cal data
- Total columns: 5 sensors, 5 * 33 columns + 2 extra columns (Annotation Level and Annotation Pulse)


In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import sys

In [3]:
path = "/home/ashish/Results/Datasets/Shimmer/Rowing/RawSignal/"
cleaned_data_path = "/home/ashish/Results/Datasets/Shimmer/Rowing/CleanedSignal/"

In [4]:
list_of_files = os.listdir(path)

In [5]:
cols_each_file = [(i, pd.read_csv(os.path.join(path, i), sep="\t").shape[1]) for i in list_of_files]

In [8]:
kw = ["Batt_Percentage", "Packet_Reception_Rate_Current", "Packet_Reception_Rate_Trial", "Timestamp", "Level", "Pulse", "_WR", "Event_Marker"]

In [ ]:
d_mapping = {"pid":[], "col_names": []}
for i in list_of_files:
    df = pd.read_csv(os.path.join(path, i), sep="\t")
    df.dropna(how='all', axis=1, inplace=True)
    uncal_columns = []
    unn_columns = []
    for c in df.columns:
        second_row = df.loc[1, c]
        first_row = df.loc[0, c]
        if type(second_row) == str and  "UNCAL" in second_row:
            uncal_columns.append(c)
        else:
            for k in kw:
                if type(first_row) == str and k in first_row:
                    unn_columns.append(c)
    df.drop(uncal_columns + unn_columns, axis=1, inplace=True)
    pid = i[:-4].split(" ")[0]
    exercise_type = i[:-4].split(" ")[-1]
    d_mapping["pid"].append("{}_{}".format(pid, exercise_type))
    
    # print(i)
    df = df.drop([1,2])
    new_column_names = []
    for c in df.columns:
        imu_name = c.split(".")[0]
        first_row = df.loc[0, c]
        combined_name = imu_name + "_" + first_row
        new_column_names.append(combined_name)
    df.columns = new_column_names
    df = df.drop([0])
    d_mapping["col_names"].append(list(df.columns))
    df.to_csv(os.path.join(cleaned_data_path, i), index=False)
    print(df.shape)
    # print(df.columns)
    

In [11]:
df.head()

,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
3,-3.168674698795181,9.698795180722891,-0.783132530120482,-0.916030534351145,6.1679389312977095,1.3893129770992365,-0.02,-0.9554545454545454,0.8275510204081633,-0.7228915662650602,...,0.5836734693877551,-0.43373493975903615,9.843373493975903,-2.216867469879518,1.816793893129771,-2.8702290076335877,2.8702290076335877,0.22181818181818183,0.012727272727272728,-0.43775510204081636
4,-3.0481927710843375,9.771084337349398,-0.7951807228915663,-1.2519083969465647,7.816793893129771,1.4198473282442747,-0.017272727272727273,-0.9536363636363636,0.8255102040816328,-0.8433734939759037,...,0.5806122448979593,-0.46987951807228917,9.843373493975903,-2.204819277108434,1.3893129770992365,0.9007633587786259,2.015267175572519,0.22363636363636363,-0.0018181818181818182,-0.43673469387755104
5,-3.204819277108434,9.698795180722891,-0.8433734939759037,-2.4732824427480917,6.778625954198473,1.1755725190839694,-0.02090909090909091,-0.9572727272727273,0.8244897959183675,-0.8313253012048193,...,0.5826530612244899,-0.6385542168674699,9.91566265060241,-2.180722891566265,1.297709923664122,3.068702290076336,0.9618320610687022,0.22363636363636363,-0.013636363636363636,-0.43673469387755104
6,-3.289156626506024,9.795180722891567,-0.8433734939759037,-1.4198473282442747,7.221374045801526,0.7480916030534351,-0.02090909090909091,-0.9527272727272728,0.826530612244898,-0.8433734939759037,...,0.5836734693877551,-0.6385542168674699,10.012048192771084,-2.204819277108434,1.1908396946564885,-0.8396946564885496,1.2213740458015268,0.22363636363636363,0.008181818181818182,-0.4357142857142858
7,-3.240963855421687,9.710843373493976,-0.746987951807229,-3.0076335877862594,5.7251908396946565,0.732824427480916,-0.017272727272727273,-0.9545454545454546,0.8244897959183675,-0.7710843373493976,...,0.5816326530612246,-0.27710843373493976,9.975903614457831,-2.325301204819277,-0.12213740458015267,-2.900763358778626,1.6335877862595418,0.22090909090909092,-0.014545454545454545,-0.4408163265306123


In [13]:
mapping_df = pd.DataFrame(d_mapping["col_names"])

In [14]:
mapping_df.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
1,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
2,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
3,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
4,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z


In [15]:
mapping_df["pid"] = d_mapping["pid"]

In [17]:
mapping_df.shape

(266, 46)

In [16]:
mapping_df.to_csv("./mapping_cols_Rowing.csv", index=False, header=None)